<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/04_Bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bootstrap

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import random

Import cleaned version of colexification dataframe from Drive

In [2]:
!gdown --id 17MaPucmlRvDR_WQ9Slgt95T7i6HRWi4- #df_colexifications

Downloading...
From: https://drive.google.com/uc?id=17MaPucmlRvDR_WQ9Slgt95T7i6HRWi4-
To: /content/df_colexifications.csv
7.62MB [00:00, 46.5MB/s]


In [4]:
df_colex = pd.read_csv('df_colexifications.csv')

In [5]:
df_colex.groupby('Family').size()

Family
Abkhaz-Adyge     302
Abun               3
Afro-Asiatic    2653
Aikanã            21
Ainu              70
                ... 
Yuat               6
Yukaghir         168
Yámana           129
Zamucoan         175
Zuni             145
Length: 181, dtype: int64

**knowing the data**: 181 linguistic fam

Add concepticon_pair column for future purposes

In [6]:
df_colex['Concepticon_pair']=list(zip(df_colex['Concepticon_Gloss.x'],df_colex['Concepticon_Gloss.y']))

In [7]:
df_colex

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Family,variety,Concepticon_ID.y,Concepticon_Gloss.y,Concepticon_pair
0,s@,1369,hrus1242,GOLD,Hruso,Hruso Aka Jamiri,1927,BAMBOO,"(GOLD, BAMBOO)"
1,s@,1369,hrus1242,GOLD,Hruso,Hruso Aka Jamiri,946,BLOOD,"(GOLD, BLOOD)"
2,avir@,1035,miji1239,GOOD,Sino-Tibetan,Dammai Dibin,923,LOVE,"(GOOD, LOVE)"
3,dzju,1425,hrus1242,GREEN,Hruso,Hruso Aka Jamiri,1424,YELLOW,"(GREEN, YELLOW)"
4,lah,1277,dakp1242,HAND,Sino-Tibetan,Monpa Changprong,639,MOUNTAIN,"(HAND, MOUNTAIN)"
...,...,...,...,...,...,...,...,...,...
125026,or,688,bong1291,LUNG,Nuclear Trans New Guinea,bongu,1223,HEART,"(LUNG, HEART)"
125027,eni,763,gira1247,SKIN,Nuclear Trans New Guinea,girawa,1405,NAME,"(SKIN, NAME)"
125028,wus,763,payn1244,SKIN,Nuclear Trans New Guinea,paynamar,51,SORE,"(SKIN, SORE)"
125029,su,51,kein1239,SORE,Nuclear Trans New Guinea,kein,1257,WING,"(SORE, WING)"


In [8]:
# df_fam --> df with families and varieties
df_fam = df_colex[['Family','Glottocode']].copy()

In [9]:
df_fam=df_fam.drop_duplicates()


In [10]:
dicFam = df_fam.groupby(['Family']).size().to_dict() # dictionary with number of variety for future purposes

### BOOTSTRAP function

In [11]:
import time

def colex_freq(x, suppDf):
  # x = df of only one colex_pair
  sumNumVar = sum(x['Num_var'])
  div = sumNumVar / 183
  newEl = [x['Colex_pair'].iloc[0], div]
  suppDf.append(newEl)


def concept_sum(x, suppDf):
  # x = df divided by concepticon inside only one family
  totLang = len(x['Glottocode']) # how many languages have that colexification faking to have different families from bootstrap --> NO SET
  # NB we don't have repetitions of Concepticon_pair in a variety, if we have it, it's just because we did the bootstrap with replacement
  newEl = [x['Family'].iloc[0], x['Concepticon_pair'].iloc[0], totLang]
  suppDf.append(newEl)


def colex_rep(x, suppDf):
  # x = df divided by family
  x.groupby('Concepticon_pair').apply(lambda y: concept_sum(y, suppDf))


def bootstrapping(cicle):
  # cicle = number of repetitions
  print ("Running bootstrapping function")
  start_time = time.time()

  listBoot = []
  
  for k in range (cicle):
    # create new df with random varieties
    new_df_fam = df_fam.groupby(['Family']).apply(lambda x: x.sample(n=(len(x['Glottocode'])), replace=True)).reset_index(drop = True)
    # create list with varieties chosen
    listVar=new_df_fam.Glottocode.tolist()
    print("List of linguistic varieties chosen")
    # new df
    boot_df = pd.DataFrame()

    for v in listVar:
      # for every variety, take all colexification and put them into boot_df
      boot_df=boot_df.append(df_colex.loc[df_colex['Glottocode'] == v], ignore_index = True)

    suppDf = [] # new empty list

    print("Taking families grouped by concepticon pair and counting number of varieties with the same colexification")

    boot_df.groupby(['Family']).apply(lambda x: colex_rep(x, suppDf))
    print("Created dataframe with colexification from chosen varieties")
    # new df from list obtained by functions
    df_sum = pd.DataFrame(suppDf, columns=['Family', 'Colex_pair', 'Var_rep'])

    df_sum['Num_var']='' # new empty column

    for i in range(len(df_sum['Colex_pair'])):
      fam = df_sum['Family'].iloc[i] # take the family for the row
      rep = df_sum['Var_rep'].iloc[i] # take var rep
      div = rep / dicFam[fam]
      df_sum['Num_var'].iloc[i] = div

    suppDf = [] # empty the list

    print("Counting colex frequency among all families")
    df_sum.groupby(['Colex_pair']).apply(lambda x: colex_freq(x, suppDf))

    listBoot.append(suppDf) # add list to list of bootstrap
    # df_colex_freq = pd.DataFrame(suppDf, columns=['Colex_pair', 'Freq']) # to save in df
    print("Finished cicle number", k)
  print("--- %s seconds ---" % (time.time() - start_time))
  return listBoot


In [13]:
random.seed(70) # random number 
lBoot = bootstrapping(1) # number is how many cicles

Running bootstrapping function
List of linguistic varieties chosen
Taking families grouped by concepticon pair and counting number of varieties with the same colexification
Created dataframe with colexification from chosen varieties


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Counting colex frequency among all families
Finished cicle number 0
--- 103.76740455627441 seconds ---


In [14]:
len(lBoot)

1

In [15]:
df_colex_freq = pd.DataFrame(lBoot[0], columns=['Colex_pair', 'Freq']) # to save in df


In [16]:
df_colex_freq

,Colex_pair,Freq
0,"(A LITTLE, WHAT)",0.000128
1,"(ABOVE, BLACK)",0.000070
2,"(ABOVE, CLOUD)",0.000082
3,"(ABOVE, DEER)",0.000228
4,"(ABOVE, DIG)",0.005464
...,...,...
42912,"(YOUNGER SISTER, SISTER)",0.000049
42913,"(YOUNGER SISTER, SON)",0.000031
42914,"(YOUNGER SISTER, STEPDAUGHTER)",0.000455
42915,"(YOUNGER SISTER, YOUNGER BROTHER)",0.011027
